# Exercise 2: Tracking with OpenCV

In this exercise you will learn how to use OpenCV's simple [Tracking API](https://docs.opencv.org/4.6.0/d9/df8/group__tracking.html).

As a first step, let's import the python modules we need.

In [1]:
import cv2 as cv
import numpy as np

Previous OpenCV releases included many trackers, but most of them have been removed in subsequent releases. The current OpenCV release (4.5.2), supports two official trackers (KCF and CSRT) and one unofficial tracker (MIL):
- MIL is the baseline tracker. It is quite slow and doesn’t work very well.
- [KCF](https://docs.opencv.org/4.6.0/d2/dff/classcv_1_1TrackerKCF.html) stands for *Kernelized Correlation Filters*. It is faster than MIL, but doesn't handle full occlusion well.
- [CSRT](https://docs.opencv.org/4.6.0/d2/da2/classcv_1_1TrackerCSRT.html) stands for *Discriminative Correlation Filter*. It is more accurate than KCF, but slightly slower.

We use OpenCV's [Tracking API](https://docs.opencv.org/4.6.0/d9/df8/group__tracking.html) as described delow.

1. Firstly, we create the desired tracking object, i.e. using one of the following functions:

> `tracker = cv.TrackerMIL_create()`  
> `tracker = cv.TrackerKCF_create()`  
> `tracker = cv.TrackerCSRT_create()`

2. Once we have a `tracker` object, we initialize it with the first `frame` and a bounding box `bbox` for the object we want to track. We do this by calling the `tracker` object's `init()` method, passing in the `frame` and `bbox` as parameters. The `init()` method's return value `ok` will be `True` if the bounding box `bbox` fits within the `frame`, otherwise `False`.

> `ok = tracker.init(frame, bbox)`

3. Finally, for each subsequent `frame`, we update the `tracker`, by calling its `update()` method, passing in the current `frame`. The `update()` method's first return value `ok` will be `True` if the `tracker` was able to track the chosen object from the previous frame to the current frame, otherwise `False`. If successfull, the second return value `bbox` contains the new bounding box.

> `ok, bbox = tracker.update(frame)`

---

Let's load in a video, and try this.

In [2]:
# Set tracking algorithm
tracker_types = ['MIL','KCF', 'CSRT']
tracker_type = tracker_types[1] # Select the KCF tracker (the fastest of the 3 trackers)
if tracker_type == 'MIL':
    tracker = cv.TrackerMIL_create()
elif tracker_type == 'KCF':
    tracker = cv.TrackerKCF_create()
elif tracker_type == "CSRT":
    tracker = cv.TrackerCSRT_create()

# Read in a video
video = cv.VideoCapture("../data/chaplin.mp4")

# Exit if we failed to load the video.
if not video.isOpened():
    print("Could not open video")

# Read the first frame.
ok, frame = video.read()
if not ok:
    print('Cannot read video file')

# Let the user select a bounding box in the video frame using the mouse.
# We can do this by using the OpenCV function "cv.selectROI()".
# When the first frame appears in a window, do the following:
# 1. Draw a bounding box around an object using your mouse.
# 2. Hit the <Enter> key to confirm your selection.
bbox = cv.selectROI("Tracking", frame, False, False)

# If the user failed to select a bounding box ...
if bbox == (0, 0, 0, 0):
    # ... define a default bounding box
    bbox = (162, 17, 116, 343)
    print('Please select a ROI')

# Initialize the tracker with the first frame and the bounding box
ok = tracker.init(frame, bbox)

while video.isOpened():
    # Read a new frame
    ok, frame = video.read()
    
    # Exit the loop, if we have reached the end of the video stream
    if not ok:
        break
    
    # Let's measure the frame rate (Frames Per Second - FPS)
    # we get when tracking the object from frame to frame
    timer = cv.getTickCount()
    
    # Update tracker with the current frame
    # This returns failure/success and the new bounding box
    ok, bbox = tracker.update(frame)
    
    # Calculate Frames per second (FPS)
    fps = cv.getTickFrequency() / (cv.getTickCount() - timer)
    
    # Draw the bounding box on the frame
    if ok:
        # Tracking success (i.e. we found the object/bounding box in the current frame)
        p1 = (int(bbox[0]), int(bbox[1]))                       # top-left point of the bounding box
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))   # bottom-right point of the bounding box
        cv.rectangle(frame, p1, p2, (255,0,0), 2, 1)            # draw the bounding box
    else:
        # Tracking failure (i.e. we did not find the object/bounding box in the current frame)
        # Let's draw an error message on the frame.
        cv.putText(frame, "Tracking failure detected", (100,80), cv.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    
    # Display the type of tracker (MIL, KCF or CSRT) on the frame
    cv.putText(frame, tracker_type + " Tracker", (100,20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
    
    # Display the frame rate (FPS) on the frame
    cv.putText(frame, "FPS : " + str(int(fps)), (100,50), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
    
    # Display the annotated frame
    cv.imshow("Tracking", frame)
    
    # Exit if the ESC key was pressed
    k = cv.waitKey(1) & 0xff
    if k == 27 : break

# Release the VideoCapture and destroy any windows we ahve created
video.release()
cv.destroyAllWindows()